In [1]:
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import cmasher as cmr
import cmocean
import cartopy.crs as ccrs


In [2]:
os.chdir("/home/548/eh6215/Desktop/python/")
from LagrangianCoherence.LCS import LCS
from LagrangianCoherence.LCS import trajectory


In [3]:
path = "/g/data/tp28/dev/barpa/prod/eh6215//cg282_ACCESS-CM2_historical_1960_sciB/19800401T0000Z/nc/"
fileend = "-CMIP6-ACCESS-CM2-historical-r4i1p1f1-barpa_r-v1-{time}.nc"

varlist = {"av_mois_flux_v":("SLV3H",1,"198004220130-198004222230"),
           "av_mois_flux_u":("SLV3H",1,"198004220130-198004222230"),
           "ttl_col_q":("SLV1H",3,"198004220100-198004230000")}

In [4]:
data={}
for var in varlist:
    stream,k,date = varlist[var]
    data[var]=xr.load_dataset(path+stream+'/'+var+fileend.format(time=date))[var]

In [5]:
q=data["ttl_col_q"][:].resample(time='3H',loffset='1.5H').interpolate('linear')
u = data["av_mois_flux_u"]/q
v = data["av_mois_flux_v"]/q


In [6]:
u.name = "u"
v.name = "v"
ds = xr.merge([u,v])

ds = ds.reset_coords(drop=True)

In [ ]:
x_dye, y_dye = trajectory.parcel_propagation(ds.u, ds.v,
                                     timestep=-3 * 3600,
                                     propdim='time',
                                     SETTLS_order=4,
                                     copy=True,
                                     # C=wv,
                                     # Srcs=evap,
                                     return_traj=True,
                                     cyclic_xboundary=False)


Propagating time 325290600000000000
SETTLS iteration 0
SETTLS iteration 1
SETTLS iteration 2
SETTLS iteration 3
Propagating time 325279800000000000
SETTLS iteration 0
SETTLS iteration 1
SETTLS iteration 2
SETTLS iteration 3
Propagating time 325269000000000000


In [ ]:
x, y = trajectory.parcel_propagation(ds.u, ds.v,
                                     timestep=3 * 3600,
                                     propdim='time',
                                     SETTLS_order=2,
                                     copy=True,
                                     # C=wv,
                                     # Srcs=evap,
                                     return_traj=True,
                                     cyclic_xboundary=False)


In [ ]:
rcs = LCS.LCS(timestep=3 * 3600, timedim='time', SETTLS_order=4 )
ftle_r = rcs(ds.copy(), isglobal=False)
ftle_r =np.log(ftle_r) / 2
ftle_r = ftle_r.where(~xr.ufuncs.isnan(ftle_r), 0)


In [ ]:
acs = LCS.LCS(timestep=-3 * 3600, timedim='time', SETTLS_order=4, )
ftle_a = acs(ds.copy(), isglobal=False, )
ftle_a = np.log(ftle_a) / 2


In [ ]:
plt.figure()
ax=plt.subplot(111,projection=ccrs.PlateCarree(0))
(y_dye.isel(time=0) - y_dye.isel(time=-1)).plot.contourf(levels=20)
ax.coastlines()
ax.set_xlim(90,180)
plt.figure()
ax=plt.subplot(111,projection=ccrs.PlateCarree(0))
(x_dye.isel(time=-1) ).plot.contourf(levels=20)
ax.coastlines()
ax.set_xlim(90,180)
plt.figure()
ax=plt.subplot(111,projection=ccrs.PlateCarree(0))
ftle_a.isel(time=0).plot.contourf(levels=20,vmin=0,vmax=2,cmap='viridis')
ds2.isel(time=0).plot.quiver('longitude', 'latitude', 'u', 'v')

ax.coastlines()
ax.set_xlim(90,180)
ax.set_ylim(-55,15)


In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={'projection': ccrs.PlateCarree()}, figsize=[8, 8])
np.sqrt(u**2 + v**2).isel(time=0).plot.contourf(levels=20, ax=ax, cmap='cubehelix')
k=15
ds2 = xr.merge([u[:,::k,::k], v[:,::k,::k]])

ds2.isel(time=0).plot.quiver('longitude', 'latitude', 'u', 'v')


ax.set_title('Wind speed (m/s)')
ax.set_xlim(90,180)
ax.coastlines(color='white')


In [ ]:
fig, axs = plt.subplots(1, 3, subplot_kw={
    'projection': ccrs.PlateCarree(central_longitude=0)}, figsize=[16, 4])
ftle_a.isel(time=0).plot(cmap=cmocean.cm.dense_r, ax=axs[0], transform=ccrs.PlateCarree(), vmin=0)
ftle_r.isel(time=0).plot(cmap=cmocean.cm.matter_r, ax=axs[1], transform=ccrs.PlateCarree(), vmin=0)
(ftle_a.isel(time=0).where(ftle_a.isel(time=0)>0,0).drop('time') -
 ftle_r.where(ftle_r.isel(time=0)>0,0).isel(time=0).drop('time')).plot(
    levels=30, cmap='bwr', ax=axs[2], transform=ccrs.PlateCarree(),vmin=-2,vmax=2)
axs[0].set_title('Attracting FTLE')
axs[1].set_title('Repelling FTLE')
axs[2].set_title('Attracting - Repelling FTLE')
axs[0].coastlines(color='white')
axs[1].coastlines(color='white')
axs[2].coastlines(color='black')


for i in range(3):
    axs[i].set_xlim(90,180)
    axs[i].set_ylim(-55,15)


In [26]:
ftle_a

<xarray.DataArray (time: 1, latitude: 360, longitude: 721)>
array([[[ 1.97597088,  1.01263049,  0.0284187 , ...,  0.87080975,
          1.9936881 ,  0.05609384],
        [ 2.21220442,  1.15378968, -0.34112869, ...,  1.18740938,
          2.2081308 , -0.24414486],
        [ 2.25937152,  1.22160882, -0.30648938, ...,  1.21981307,
          2.25958526,  0.09666402],
        ...,
        [ 2.9301434 ,  1.89846133,  1.16728205, ...,  1.90868768,
          2.92881557,  0.92169566],
        [ 3.09612502,  2.05277243,  0.63815857, ...,  2.06119872,
          3.09506524,  0.6331469 ],
        [ 0.63410442,  2.51541809,  1.48971679, ...,  0.6402165 ,
          1.48965158,  2.51520786]]])
Coordinates:
  * time       (time) datetime64[ns] 1980-04-22T01:30:00
  * latitude   (latitude) float64 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * longitude  (longitude) float64 -180.0 -179.5 -179.0 ... 178.5 179.0 179.5

In [31]:
acs

NameError: name 'ss' is not defined